In [1]:
import os
import json
from lxml import html
from pathlib import Path

In [2]:
tgtwd = 'C:\\Users\\ashbu\\Desktop\\Personal Projects\\Parler Data Merge\\'

In [3]:
post_path = tgtwd +  "\parler_2020-01-06_posts-partial\posts"

In [4]:
%%time
files = os.listdir(post_path)

CPU times: total: 2.97 s
Wall time: 3.62 s


In [5]:
json_out = open("ddos_posts.ndjson", "w") # opening our file for the json output

In [7]:
%%time

# looping through all the files
for file in files:
    
    # grabbing the file text from the file
    file_text = Path(post_path+"\\"+file).read_text(encoding="utf-8")
    
    # loading the file text into lxml to parse it
    tree = html.fromstring(file_text)
    
    # making a dict to hold everything
    post = dict()
    
    # the filename
    post["filename"] = file    
    
    # who echoed it (fixed)
    post["echoed_by"] = tree.xpath('//div[contains(@class, "eb--col eb--statement")]//span[contains(@class, "reblock")]/text()')

    # when it was echoed (fixed)
    post["when_echoed"] = tree.xpath('//div[contains(@class, "eb--col eb--timestamp")]//span[contains(@class, "reblock")]/text()')
    
    # timestamp (fixed)
    post["timestamp"] = tree.xpath('//span[contains(@class, "post--timestamp")]/text()')

    # author (fixed)
    post["author"] = tree.xpath('//span[contains(@class, "author--username")]/text()')

    # I need to add a check for if there is no text
    # post content -> confirm works with single p tag -> we'll have to figure out how it handles more though
    post["content"] = tree.xpath('//div[contains(@class, "card--body")]//p/text()')

    # impressions (fixed)
    post["impressions"] = tree.xpath('//span[contains(@class, "impressions--count")]/text()')

    # post comments (fixed)
    post["comments"] = tree.xpath('//span[preceding-sibling::img[contains(@alt, "Post Comments")]]/text()')

    # post echoes (fixed)
    post["echoes"] = tree.xpath('//span[preceding-sibling::img[contains(@alt, "Post Echoes")]]/text()')

    # post upvotes (fixed)
    post["upvotes"] = tree.xpath('//span[preceding-sibling::img[contains(@alt, "Post Upvotes")]]/text()')
         
    # next we need to determine if the dict is empty or not
    # if empty, then we write the html of the file text to a file to track that and increment the counter there
    # harder part tbh, but I think well worth the effort given what we're doing here
    
    # else, we'll write the dict after converting to JSON
    else:
        json_out.write(json.dumps(post) + "\n") # dumping the output

CPU times: total: 44min 14s
Wall time: 2h 29min 58s


In [8]:
json_out.close()

In [9]:
import pandas as pd

In [10]:
# checking the writing was successful
pd.read_json("ddos_posts.ndjson", lines=True)

,filename,echoed_by,when_echoed,timestamp,author,content,impressions,comments,echoes,upvotes
0,$00003d96-5210-462c-af2e-1febf03fc7be,[],[],[],[],[],[],[],[],[]
1,$0005eaf1-6e14-4632-91c1-d488b1a97207,[],[],[],[],[],[],[],[],[]
2,$00097a6b-475f-4fd3-8191-8eb4d668e9e6,[],[],[],[],[],[],[],[],[]
3,$0009a444-a8f8-4d66-905a-9156ee02acdc,[],[],[],[],[],[],[],[],[]
4,$000b97cc-4c77-4e27-bcdd-c5764c1bfad5,[],[],[],[],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...
1854325,fffff887566e4e28a0e4ecd3be9f4ab6,[Echoed By Kee],[4 days ago],[4 days ago],[@Martycos],[Here It Is: Italy And Obama’s Orchestration o...,[7557],[16],[121],[110]
1854326,fffffb044223456eb54eb5a0931c91d1,[Echoed By Jadarwardrealtor],[4 days ago],[5 days ago],[@QanonMemes],[🚨OBAMAGATE AUDIO LEAK: Barack Obama and The M...,[76444],[100],[630],[815]
1854327,fffffd150701457b92d80fc04f1cba5f,[Echoed By Esamilpa],[4 days ago],[4 days ago],[@linwood],[Discernment.],[12397137],[2118],[21151],[48177]
1854328,_html_,[],[],[],[],[],[],[],[],[]
